In [1]:
import duckdb
import pandas as pd

In [2]:
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

There's a new jupysql version available (0.10.9), you're running 0.10.3. To upgrade: pip install jupysql --upgrade
Deploy AI and data apps for free on Ploomber Cloud! Learn more: https://docs.cloud.ploomber.io/en/latest/quickstart/signup.html


## EXTRACT

In [3]:
# EXTRACT
%sql CREATE TABLE raw AS SELECT * FROM read_json_auto('../dataset/parts/house*', format='newline_delimited')

Running query in 'duckdb'

Count
20146


In [4]:
%sql SELECT * FROM raw LIMIT 1

Running query in 'duckdb'

id,price,installment,address,tags,description,specs,facilities,agent,images,url,last_modified,scraped_at
hos7134669,3300.0,14.0,"Puncak, Bogor",[],"Di jual rumah dengan suasana aman dan nyaman jalan lebar bagus, berada di vimala hills puncak dengan udara yang sejuk dan bersih. kondisi rumah baru, bagus dan rapi siap huni, harga masih bisa nego fasilitas lengkap sehingga memudahkan anda untuk beraktifitas sehari-hari. kamar tidur utama 1, kamar tidur tamu 2, kamar pembantu, gudang, toilet utama, toilet tamu, toilet ART, gazeboo diatas garasi/carport. untuk info selanjutnya silahkan hubungi Senior Marketing kamiDESSY : 0818 0792 1xxx (wa)","{'Kamar Tidur': 3, 'Kamar Mandi': 3, 'Luas Tanah': '300 m²', 'Luas Bangunan': '180 m²', 'Tipe Properti': 'Rumah', 'Sertifikat': 'SHM - Sertifikat Hak Milik', 'Daya Listrik': '2200 Watt', 'Kamar Pembantu': 1, 'Kamar Mandi Pembantu': 1, 'Jumlah Lantai': 1, 'Tahun Dibangun': 1000, 'Kondisi Properti': 'Bagus Sekali', 'ID Iklan': 'hos7134669', 'Dapur': None, 'Ruang Makan': None, 'Ruang Tamu': None, 'Kondisi Perabotan': None, 'Material Bangunan': None, 'Material Lantai': None, 'Garasi': None, 'Pemandangan': None, 'Terjangkau Internet': None, 'Lebar Jalan': None, 'Hook': None, 'Carport': None, 'Hadap': None, 'Sumber Air': None, 'Tahun Di Renovasi': None, 'Konsep dan Gaya Rumah': None}",[],"{'name': 'Dessy Nine', 'url': 'https://www.rumah123.com/agen-properti/nine-property/dessy-nine-9597/', 'phone': '+6281807921550', 'company': {'name': 'Nine Property', 'url': 'https://www.rumah123.com/agen-properti/nine-property-1589/'}}",[],https://www.rumah123.com/properti/bogor/hos7134669/,2024-01-18T00:00:00,2024-01-21T03:10:51.029727


In [5]:
conn.sql("DESCRIBE raw")

┌───────────────┬────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬───────┐
│  column_name  │                          column_type                           │  null   │   key   │ default │ extra │
│    varchar    │                            varchar                             │ varchar │ varchar │ varchar │ int32 │
├───────────────┼────────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼───────┤
│ id            │ VARCHAR                                                        │ YES     │ NULL    │ NULL    │  NULL │
│ price         │ DOUBLE                                                         │ YES     │ NULL    │ NULL    │  NULL │
│ installment   │ DOUBLE                                                         │ YES     │ NULL    │ NULL    │  NULL │
│ address       │ VARCHAR                                                        │ YES     │ NULL    │ NULL    │  NULL │
│ tags          │ VARCHAR[]     

## TRANSFORM

### Transform 1 - Specs

In [6]:
# TRANSFORM 1 - Splitting the specs column into multiple columns
%sql CREATE OR REPLACE TABLE house_specs_raw AS SELECT specs.*, id AS reference_id FROM raw

Running query in 'duckdb'

Count
20146


In [7]:
cols = conn.sql("DESCRIBE house_specs_raw").df()["column_name"].values
cols = [f"\"{col}\" AS {col.lower().replace(' ', '_')}" for col in cols if col != "reference_id"] + ["reference_id"]
conn.sql(f"CREATE OR REPLACE TABLE house_specs AS SELECT {', '.join(cols)} FROM house_specs_raw")

In [8]:
%sql SELECT * FROM house_specs LIMIT 2

Running query in 'duckdb'

kamar_tidur,kamar_mandi,luas_tanah,luas_bangunan,tipe_properti,sertifikat,daya_listrik,kamar_pembantu,kamar_mandi_pembantu,jumlah_lantai,tahun_dibangun,kondisi_properti,id_iklan,dapur,ruang_makan,ruang_tamu,kondisi_perabotan,material_bangunan,material_lantai,garasi,pemandangan,terjangkau_internet,lebar_jalan,hook,carport,hadap,sumber_air,tahun_di_renovasi,konsep_dan_gaya_rumah,reference_id
3,3,300 m²,180 m²,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,1,1,1,1000,Bagus Sekali,hos7134669,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,hos7134669
4,3,385 m²,270 m²,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,4,1,1,2016,Bagus,hos7134732,1,Tidak,Tidak,Semi Furnished,Bata Merah,Keramik,4,Pegunungan,Ya,2 Mobil,Tidak,None,None,None,None,None,hos7134732


In [9]:
%sql SELECT COUNT(*), lebar_jalan FROM house_specs GROUP BY lebar_jalan

Running query in 'duckdb'

count_star(),lebar_jalan
409,4 Mobil
7721,None
1336,1 Mobil
9212,2 Mobil
1468,3 Mobil


In [10]:
%sql CREATE OR REPLACE TABLE lebar_jalan AS SELECT split_part(lebar_jalan, ' ', 1) AS lebar_jalan_num FROM house_specs

Running query in 'duckdb'

Count
20146


In [11]:
%sql SELECT COUNT(*), lebar_jalan_num FROM lebar_jalan GROUP BY lebar_jalan_num

Running query in 'duckdb'

count_star(),lebar_jalan_num
7721,
1336,1
9212,2
1468,3
409,4


In [12]:
conn.sql("DESCRIBE house_specs").df()

,column_name,column_type,null,key,default,extra
0,kamar_tidur,BIGINT,YES,None,None,NaN
1,kamar_mandi,BIGINT,YES,None,None,NaN
2,luas_tanah,VARCHAR,YES,None,None,NaN
3,luas_bangunan,VARCHAR,YES,None,None,NaN
4,tipe_properti,VARCHAR,YES,None,None,NaN
5,sertifikat,VARCHAR,YES,None,None,NaN
6,daya_listrik,VARCHAR,YES,None,None,NaN
7,kamar_pembantu,BIGINT,YES,None,None,NaN
8,kamar_mandi_pembantu,BIGINT,YES,None,None,NaN
9,jumlah_lantai,BIGINT,YES,None,None,NaN


In [13]:
# parse luas_tanah, luas_bangunan, daya_listrik, lebar_jalan as numeric
%sql CREATE OR REPLACE TABLE house_specs AS SELECT *, TRY_CAST(RTRIM(luas_tanah, ' m²') AS FLOAT) AS luas_tanah_num, TRY_CAST(RTRIM(luas_bangunan, ' m²') AS FLOAT) AS luas_bangunan_num, TRY_CAST(REPLACE(RTRIM(LOWER(daya_listrik), 'watt'), 'lainnya', '0') AS FLOAT) AS daya_listrik_num, TRY_CAST(SPLIT_PART(lebar_jalan, ' ', 0) AS FLOAT) AS lebar_jalan_num FROM house_specs

# drop luas_tanah, luas_bangunan, daya_listrik, lebar_jalan
%sql ALTER TABLE house_specs DROP luas_tanah
%sql ALTER TABLE house_specs DROP luas_bangunan
%sql ALTER TABLE house_specs DROP daya_listrik
%sql ALTER TABLE house_specs DROP lebar_jalan

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Success


In [14]:
# normalize kondisi_properti
%sql CREATE OR REPLACE TABLE house_specs AS SELECT *, UPPER(kondisi_properti) AS kondisi_properti_norm, UPPER(kondisi_perabotan) AS kondisi_perabotan_norm FROM house_specs

# drop kondisi_properti, kondisi_perabotan
%sql ALTER TABLE house_specs DROP kondisi_properti
%sql ALTER TABLE house_specs DROP kondisi_perabotan

# normalize kondisi_properti
%sql UPDATE house_specs SET kondisi_properti_norm = CASE kondisi_properti_norm WHEN 'BAGUS SEKALI' THEN 'FURNISHED' WHEN 'SUDAH RENOVASI' THEN 'FURNISHED' WHEN 'BUTUH RENOVASI' THEN 'UNFURNISHED' WHEN 'BAGUS' THEN 'FURNISHED' WHEN 'BARU' THEN 'FURNISHED' ELSE kondisi_properti_norm END
%sql UPDATE house_specs SET kondisi_perabotan_norm = CASE kondisi_perabotan_norm WHEN 'BAGUS SEKALI' THEN 'FURNISHED' WHEN 'SUDAH RENOVASI' THEN 'FURNISHED' WHEN 'BUTUH RENOVASI' THEN 'UNFURNISHED' WHEN 'BAGUS' THEN 'FURNISHED' WHEN 'BARU' THEN 'FURNISHED' ELSE kondisi_perabotan_norm END

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Count
20146


In [15]:
# convert to boolean
%sql CREATE OR REPLACE TABLE house_specs AS SELECT *, ruang_makan = 'Ya' AS ruang_makan_available, ruang_tamu = 'Ya' AS ruang_tamu_available, terjangkau_internet = 'Ya' AS terjangkau_internet_available, hook = 'Ya' AS hook_available FROM house_specs

# drop ruang_makan, ruang_tamu, terjangkau_internet, hook
%sql ALTER TABLE house_specs DROP ruang_makan
%sql ALTER TABLE house_specs DROP ruang_tamu
%sql ALTER TABLE house_specs DROP terjangkau_internet
%sql ALTER TABLE house_specs DROP hook

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Success


In [16]:
# split material_bangunan to new table
%sql CREATE OR REPLACE TABLE house_material AS SELECT UNNEST(string_split(UPPER(material_bangunan), ', ')) AS value, reference_id FROM house_specs
%sql CREATE OR REPLACE TABLE house_material AS SELECT replace(v)
# %sql ALTER TABLE house_specs DROP material_bangunan

Running query in 'duckdb'

Running query in 'duckdb'

BinderException: Binder Error: Referenced column "v" not found in FROM clause!
LINE 1: ...TABLE house_material AS SELECT replace(v)
                                                  ^

In [17]:
%sql SELECT COUNT(*), value FROM house_material GROUP BY value

Running query in 'duckdb'

count_star(),value
917,BATAKO
5112,BATA HEBEL
4708,BATA MERAH
1652,BETON


In [18]:
# split material_lantai to new table
%sql CREATE OR REPLACE TABLE house_floor_material AS SELECT UNNEST(string_split(UPPER(material_lantai), ', ')), reference_id FROM house_specs
%sql ALTER TABLE house_specs DROP material_lantai

Running query in 'duckdb'

Running query in 'duckdb'

Success


In [19]:
# drop id_iklan
%sql ALTER TABLE house_specs DROP COLUMN id_iklan

# drop tipe_properti because this column is constant
%sql ALTER TABLE house_specs DROP tipe_properti

Running query in 'duckdb'

Running query in 'duckdb'

Success


### Transform 2 - Tags

In [20]:
# TRANSFORM 2 - Splitting the tags column into multiple rows
%sql CREATE OR REPLACE TABLE house_tags AS SELECT UNNEST(raw.tags) AS tag, id AS reference_id FROM raw
%sql CREATE OR REPLACE TABLE house_tags AS SELECT UNNEST(string_split(tag, '/')) AS tag, reference_id FROM house_tags

Running query in 'duckdb'

Running query in 'duckdb'

Count
69632


In [21]:
%sql SELECT * FROM house_tags LIMIT 5

Running query in 'duckdb'

tag,reference_id
Bisa Nego,hos7134732
One Gate System,hos7134732
Bisa Nego,hos7253604
Bisa Nego,hos7062653
Cash Keras,hos7062653


### Transform 3 - Facilities

In [22]:
# TRANSFORM 3 - Splitting the facilities column into multiple rows
%sql CREATE OR REPLACE TABLE house_facilities AS SELECT UNNEST(raw.facilities) AS tag, id AS reference_id FROM raw

Running query in 'duckdb'

Count
137456


In [23]:
%sql SELECT * FROM house_facilities LIMIT 5

Running query in 'duckdb'

tag,reference_id
Jalur Telepon,hos7134732
Kolam Ikan,hos7134732
Taman,hos7134732
Tempat Jemuran,hos7134732
Akses Parkir,hos7134732


### Transform 4 - Images

In [ ]:
# TRANSFORM 4 - Splitting the images column into multiple rows
%sql CREATE TABLE house_images AS SELECT UNNEST(raw.images) AS url, id AS reference_id FROM raw

### Transform 5 - Agent

In [ ]:
# TRANSFORM 5 - Splitting the agent column into multiple columns
%sql CREATE OR REPLACE TABLE house_agent_raw AS SELECT agent.*, id AS reference_id FROM raw

In [ ]:
def mask_name(s: str) -> str:
    if s is None or len(s) < 1:
        return None
    
    combined = ""
    parts = s.split()
    for part in parts:
        combined += part[:1] + "x" * (len(part) - 1) + " "

    return combined.strip()

def mask_phone(s: str) -> str:
    if s is None or len(s) < 3:
        return None
    
    return ("x" * (len(s) - 3)) + s[-3:]

conn.create_function("MASK_NAME", mask_name)
conn.create_function("MASK_PHONE", mask_phone)

In [ ]:
# split agent company
%sql CREATE OR REPLACE TABLE house_agent_company AS SELECT company.*, reference_id FROM house_agent_raw

In [ ]:
# mask name and phone
%sql CREATE OR REPLACE TABLE house_agent AS SELECT mask_name(name) AS name, mask_phone(phone) AS phone, sha256(CONCAT(name, phone)) AS agent_hash, reference_id FROM house_agent_raw

### Transform 6 - Raw to Houses

In [ ]:
# TRANSFORM 6 - Project new columns
%sql CREATE OR REPLACE TABLE houses AS SELECT id, price, installment, address, description, url, last_modified AS last_modified_at, scraped_at FROM raw

## LOAD

In [ ]:
conn.sql("SHOW TABLES")

In [ ]:
# LOAD
%sql COPY houses TO '../dataset/etl/L1.houses.parquet' (FORMAT 'parquet')
%sql COPY house_specs TO '../dataset/etl/L1.house_specs.parquet' (FORMAT 'parquet')
%sql COPY house_agent TO '../dataset/etl/L1.house_agent.parquet' (FORMAT 'parquet')
%sql COPY house_images TO '../dataset/etl/L1.house_images.parquet' (FORMAT 'parquet')
%sql COPY house_material TO '../dataset/etl/L1.house_material.parquet' (FORMAT 'parquet')
%sql COPY house_specs TO '../dataset/etl/L1.house_specs.parquet' (FORMAT 'parquet')
%sql COPY house_tags TO '../dataset/etl/L1.house_tags.parquet' (FORMAT 'parquet')